In [1]:
CLASSES = ["Tomato___Bacterial_spot", 
"Tomato___Early_blight", 
"Tomato___Late_blight",
"Tomato___Leaf_Mold",
"Tomato___Septoria_leaf_spot",
"Tomato___Spider_mites Two-spotted_spider_mite",
"Tomato___Target_Spot",
"Tomato___Tomato_Yellow_Leaf_Curl_Virus",
"Tomato___Tomato_mosaic_virus",
"Tomato___healthy"]

In [2]:
import tensorflow as tf

def preprocess(img_bytes):
    img = tf.image.decode_jpeg(img_bytes, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, [224, 224])
    return img


def read_from_jpegfile(filename):
    # same code as in 05_create_dataset/jpeg_to_tfrecord.py
    img = tf.io.read_file(filename)
    img = preprocess(img)
    return img

def postprocess(pred):
    top_prob = tf.math.reduce_max(pred, axis=[1])
    pred_label_index = tf.math.argmax(pred, axis=1)
    pred_label = tf.gather(tf.convert_to_tensor(CLASSES), pred_label_index)

    # custom output
    return {
        "probability": top_prob,
        "plant_type_int": pred_label_index,
        "plant_type_str": pred_label,
    }


# fmt: off
# this function receives 1 string value.
@tf.function(input_signature=[tf.TensorSpec([None,], dtype=tf.string)])
def predict_from_filename(filenames):

    # custom pre-process
    input_images = tf.map_fn(
        read_from_jpegfile, filenames, fn_output_signature=tf.float32
    )

    # model
    batch_pred = model(input_images)  # same as model.predict()

    # custom post-process
    processed = postprocess(batch_pred)
    return processed

# fmt: on

2024-06-26 15:39:58.761950: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# fmt: off
@tf.function(input_signature=[tf.TensorSpec([None,], dtype=tf.string)])
def predict_from_b64(img_bytes):

    # custom pre-process
    input_images = tf.map_fn(
        preprocess, img_bytes, fn_output_signature=tf.float32
    )

    # model
    batch_pred = model(input_images)  # same as model.predict()

    # custom post-process
    processed = postprocess(batch_pred)
    return processed
# fmt: on

In [4]:
import tensorflow as tf

model = tf.keras.models.load_model('../model/model_mb_v2_finetuned_5_reducelr.hdf5')

# Export model with a new serving function.
model.save(
    "../model/export/model_mb_v2_finetuned_5_reducelr_with_signature",
    signatures={
        "serving_default": predict_from_filename,
        "predict_base64": predict_from_b64,
    },
)

2024-06-26 15:40:00.966142: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-26 15:40:00.969881: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-26 15:40:00.980630: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

INFO:tensorflow:Assets written to: ../model/export/model_mb_v2_finetuned_5_reducelr_with_signature/assets


INFO:tensorflow:Assets written to: ../model/export/model_mb_v2_finetuned_5_reducelr_with_signature/assets


In [7]:
!saved_model_cli show --dir ../model/export/model_mb_v2_finetuned_5_reducelr_with_signature --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['filenames'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: serving_default_filenames:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['plant_type_int'] tensor_info:
      dtype: DT_INT64
      shape: (-1)
      name: StatefulPartitionedCall_1:0
  outputs['plant_type_str'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: StatefulPartitionedCall_1:1
  outputs['probability'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1)
      name: StatefulPartitionedCall_1:2
Method name is: tensorflow/serving/predict


In [9]:
from google.cloud import storage
from collections import defaultdict
import os
import re
import random
import os

PROJECT = !gcloud config list --format 'value(core.project)'
PROJECT = PROJECT[0]
BUCKET = PROJECT+"-capstone"
REGION = "us-central1"

os.environ["BUCKET"] = BUCKET
os.environ["REGION"] = REGION

# Initialize the storage client
storage_client = storage.Client()

# Set bucket name from environment variable
bucket_name = os.environ["BUCKET"]
bucket = storage_client.bucket(bucket_name)

image_folder = "test"

# List all image files in the specified folder
blobs = bucket.list_blobs(prefix=image_folder)

image_urls = []
labels = []
images = []


# Dictionary to keep track of image counts per label
label_counts = defaultdict(int)

# Dictionary to set a random limit for each label
label_limits = defaultdict(lambda: random.randint(600, 700))

# Collect image URLs and their labels, limit to a random number between 600 and 700 per label
for blob in blobs:
    if blob.name.lower().endswith(('.png', '.jpg', '.jpeg')) and blob.name.lower().startswith('test/tomato'):
        label = blob.name.replace('test/', '')
        if label_counts[label] < label_limits[label]:
            image_urls.append(f"gs://{bucket_name}/{blob.name}")
            labels.append(label)
            label_counts[label] += 1

print(f"Found {len(image_urls)} images.")

# Print the count of images for each label
for label, count in label_counts.items():
    print(f"Label: {label}, Number of Images: {count}")


Found 16 images.
Label: TomatoEarlyBlight1.JPG, Number of Images: 1
Label: TomatoEarlyBlight2.JPG, Number of Images: 1
Label: TomatoEarlyBlight3.JPG, Number of Images: 1
Label: TomatoEarlyBlight4.JPG, Number of Images: 1
Label: TomatoEarlyBlight5.JPG, Number of Images: 1
Label: TomatoEarlyBlight6.JPG, Number of Images: 1
Label: TomatoHealthy1.JPG, Number of Images: 1
Label: TomatoHealthy2.JPG, Number of Images: 1
Label: TomatoHealthy3.JPG, Number of Images: 1
Label: TomatoHealthy4.JPG, Number of Images: 1
Label: TomatoYellowCurlVirus1.JPG, Number of Images: 1
Label: TomatoYellowCurlVirus2.JPG, Number of Images: 1
Label: TomatoYellowCurlVirus3.JPG, Number of Images: 1
Label: TomatoYellowCurlVirus4.JPG, Number of Images: 1
Label: TomatoYellowCurlVirus5.JPG, Number of Images: 1
Label: TomatoYellowCurlVirus6.JPG, Number of Images: 1


In [11]:
serving_fn = tf.keras.models.load_model(
    "../model/export/model_mb_v2_finetuned_5_reducelr_with_signature"
).signatures["serving_default"]

pred = serving_fn(tf.convert_to_tensor(image_urls))
# print custom outputs
for k in pred.keys():
    print(f"{k:15}: {pred[k].numpy()}")

2024-06-26 15:59:46.202160: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8900


plant_type_str : [b'Tomato___Early_blight' b'Tomato___Bacterial_spot'
 b'Tomato___Early_blight' b'Tomato___Early_blight'
 b'Tomato___Early_blight' b'Tomato___Early_blight' b'Tomato___healthy'
 b'Tomato___healthy' b'Tomato___healthy' b'Tomato___healthy'
 b'Tomato___Tomato_Yellow_Leaf_Curl_Virus'
 b'Tomato___Tomato_Yellow_Leaf_Curl_Virus'
 b'Tomato___Tomato_Yellow_Leaf_Curl_Virus'
 b'Tomato___Tomato_Yellow_Leaf_Curl_Virus'
 b'Tomato___Tomato_Yellow_Leaf_Curl_Virus'
 b'Tomato___Tomato_Yellow_Leaf_Curl_Virus']
probability    : [0.9908867  0.96336704 0.99975306 0.99343324 0.9776037  0.9999504
 0.99999464 1.         0.9842732  0.9997434  1.         1.
 1.         1.         1.         1.        ]
plant_type_int : [1 0 1 1 1 1 9 9 9 9 7 7 7 7 7 7]


In [13]:
!saved_model_cli show --dir ../model/export/model_mb_v2_finetuned_5_reducelr_with_signature --tag_set serve --signature_def predict_base64

The given SavedModel SignatureDef contains the following input(s):
  inputs['img_bytes'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: predict_base64_img_bytes:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['plant_type_int'] tensor_info:
      dtype: DT_INT64
      shape: (-1)
      name: StatefulPartitionedCall:0
  outputs['plant_type_str'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: StatefulPartitionedCall:1
  outputs['probability'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1)
      name: StatefulPartitionedCall:2
Method name is: tensorflow/serving/predict


In [18]:
from datetime import datetime
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

REGION = "us-central1"
MODEL_DISPLAYNAME = f"model_mb_v2_finetuned_5_reducelr-{TIMESTAMP}"

print(f"MODEL_DISPLAYNAME: {MODEL_DISPLAYNAME}")

# from https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers
SERVING_CONTAINER_IMAGE_URI = (
    "us-docker.pkg.dev/vertex-ai/prediction/tf2-gpu.2-11:latest"
)

MODEL_DISPLAYNAME: model_mb_v2_finetuned_5_reducelr-20240626160223


In [19]:
!gsutil cp -R ../model/export/model_mb_v2_finetuned_5_reducelr_with_signature gs://{BUCKET}/{MODEL_DISPLAYNAME}

Copying file://../model/export/model_mb_v2_finetuned_5_reducelr_with_signature/fingerprint.pb [Content-Type=application/octet-stream]...
Copying file://../model/export/model_mb_v2_finetuned_5_reducelr_with_signature/keras_metadata.pb [Content-Type=application/octet-stream]...
Copying file://../model/export/model_mb_v2_finetuned_5_reducelr_with_signature/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://../model/export/model_mb_v2_finetuned_5_reducelr_with_signature/variables/variables.index [Content-Type=application/octet-stream]...
- [4 files][  3.5 MiB/  3.5 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://../model/export/model_mb_v2_finetuned_5_reducelr_with_signature/variables/variables.data-0

In [24]:
MACHINE_TYPE = "n1-standard-4"

endpoint = uploaded_model.deploy(
    machine_type=MACHINE_TYPE,
    accelerator_type="NVIDIA_TESLA_T4",
    accelerator_count=1,
)

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/853791352780/locations/us-central1/endpoints/2952758267601747968/operations/4087578821518163968
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/853791352780/locations/us-central1/endpoints/2952758267601747968
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/853791352780/locations/us-central1/endpoints/2952758267601747968')
INFO:google.cloud.aiplatform.models:Deploying model to Endpoint : projects/853791352780/locations/us-central1/endpoints/2952758267601747968
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/853791352780/locations/us-central1/endpoints/2952758267601747968/operations/8192046931912949760
INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/85379135278

In [ ]:
# instances = [{"filenames": f} for f in image_urls]
# pred = endpoint.predict(instances=instances)

# # print custom outputs
# for p in pred.predictions:
#     for k in p.keys():
#         print(f"{k:15}: {p[k]}")
#     print("*" * 30)

In [34]:
from oauth2client.client import GoogleCredentials

token = (
    GoogleCredentials.get_application_default().get_access_token().access_token
)
headers = {"Authorization": "Bearer " + token}

In [6]:
from google.cloud import aiplatform

uploaded_model = aiplatform.Model.upload(
    display_name=MODEL_DISPLAYNAME,
    artifact_uri=f"gs://{BUCKET}/{MODEL_DISPLAYNAME}",
    serving_container_image_uri=SERVING_CONTAINER_IMAGE_URI,
)

NameError: name 'MODEL_DISPLAYNAME' is not defined

In [37]:
import base64
def b64encode(filename):
    with open(filename, "rb") as ifp:
        img_bytes = ifp.read()
        return base64.b64encode(img_bytes).decode()


data = {
    "signature_name": "predict_base64",
    "instances": [{"img_bytes": {"b64": b64encode("./sample.jpg")}}],
}

In [40]:
import requests
import json

api = "https://{}-aiplatform.googleapis.com/v1/projects/{}/locations/{}/endpoints/{}:rawPredict".format(
    REGION, PROJECT, REGION, endpoint.name
)

response = requests.post(api, json=data, headers=headers)
json.loads(response.content)

{'predictions': [{'plant_type_str': 'Tomato___Early_blight',
   'probability': 0.990886927,
   'plant_type_int': 1}]}

In [ ]:
predict_custom_trained_model_sample(
    project="853791352780",
    endpoint_id="2952758267601747968",
    location="us-central1",
    instances={ "instance_key_1": "value", ...}
)